# Langchain Agent and Tools Tutorial

In [ ]:
from langchain_openai import AzureChatOpenAI
import os

os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["AZURE_OPENAI_API_VERSION"] = os.getenv("AZURE_OPENAI_API_VERSION")  # Use the correct API version

In [22]:
# Setup azure openai connection
llm = AzureChatOpenAI(
    model="gpt-4o",
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=os.environ["AZURE_OPENAI_API_VERSION"])

# 1. Agent and Tools Basics

Example 1: accessing date time of our computer

In [ ]:
from langchain import hub
from langchain.agents import (
    AgentExecutor,
    create_react_agent
)
from langchain_core.tools import Tool

In [ ]:
# Define a very simple tool function that returns the current time
def get_current_time(*args, **kwargs):
    """
    Return the current time in H:MM AM/PM format.
    """
    import datetime 
    now = datetime.datetime.now()
    return now.strftime("%I:%M %p")

In [ ]:
# List of tools available to the agent

tools = [
    Tool(
        name="Time",
        func=get_current_time, #function the tool will execute
        #description of the tool
        description="Useful when you need to know the current time."
    ),
]

In [ ]:
# Pull the prompt template from the hub
# ReAct = Reason and Action
# https://smith.langchain.com/hub/hwchase17/react
prompt = hub.pull("hwchase17/react")

In [ ]:
# Create the ReAct agent using the create_react_agent function
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
    stop_sequence=True
)

In [ ]:
# Create an agent executor from the agent and tools
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True
)

In [ ]:
# Run the agent with a test query
response = agent_executor.invoke({"input": "What time is it?"})

In [ ]:
print("response", response)

# 2. ReAct Chat

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.tools import Tool

In [ ]:
# Define a very simple tool function that returns the current time
def get_current_time(*args, **kwargs):
    """
    Return the current time in H:MM AM/PM format.
    """
    import datetime 
    now = datetime.datetime.now()
    return now.strftime("%I:%M %p")

In [ ]:
def search_wikipedia(query):
    """Searches wikipedia and returns summary of the first result"""
    from wikipedia import summary
    try:
        # Limit to two sentences for brevity
        return summary(query, sentence=2)
    except:
        return "I couldnt find any information on that"

In [ ]:
# Define the tools that agent can use
tools = [
    Tool(
        name="Time",
        func=get_current_time,
        description="Useful for when you need to know the current time.",
    ),
    Tool(
        name="Wikipedia",
        func=search_wikipedia,
        description="Useful for when you need to know information about a topic.",
    ),
]

In [ ]:
# Load the correct JSON Chat Prompt from the hub
prompt = hub.pull("hwchase17/structured-chat-agent")


In [ ]:
# Create a structured Chat Agent with Conversation Buffer Memory
# ConversationBufferMemory stores the conversation history, allowing the agent to maintain context across interactions

memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True
)

In [ ]:
# create_structured_chat_agent initializes a chat agent designed to interact using a structured prompt and tools
# It combines the language model (llm), tools, and prompt to create an interactive agent
agent = create_structured_chat_agent(llm=llm, tools=tools, prompt=prompt)

In [ ]:
# AgentExecutor is responsible for managing the interaction between the user input, the agent, and the tools
# It also handles memory to ensure context is maintained throughout the conversation
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory,  # Use the conversation memory to maintain context
    handle_parsing_errors=True,  # Handle any parsing errors gracefully
)

In [ ]:
# Initial system message to set the context for the chat
# SystemMessage is used to define a message from the system to the agent, setting initial instructions or context
initial_message = "You are an AI assistant that can provide helpful answers using available tools.\nIf you are unable to answer, you can use the following tools: Time and Wikipedia."
memory.chat_memory.add_message(SystemMessage(content=initial_message))

In [ ]:
# # Chat Loop to interact with the user
# while True:
#     user_input = input("User: ")
#     if user_input.lower() == "exit":
#         break

#     # Add the user's message to the conversation memory
#     memory.chat_memory.add_message(HumanMessage(content=user_input))

#     # Invoke the agent with the user input and the current chat history
#     response = agent_executor.invoke({"input": user_input})
#     print("Bot:", response["output"])

#     # Add the agent's response to the conversation memory
#     memory.chat_memory.add_message(AIMessage(content=response["output"]))


# 3. ReAct RAG

In [4]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import Chroma
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import Tool
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [5]:
current_dir = os.path.abspath("")
books_dir = os.path.join(current_dir, "books") # Earlier he just described one file with an extra argument. Now its entire folder.
db_dir = os.path.join(current_dir, "db")
persistent_directory = os.path.join(db_dir, "chroma_db_with_metadata") # Earlier it was just chroma but now its chroma with metadata


In [6]:
# Check if the Chroma vector store already exists
if os.path.exists(persistent_directory):
    print("Loading existing vector store...")
    db = Chroma(persist_directory=persistent_directory,
                embedding_function=None)
else:
    raise FileNotFoundError(
        f"The directory {persistent_directory} does not exist. Please check the path."
    )


Loading existing vector store...


C:\Users\Siddharth\AppData\Local\Temp\ipykernel_32664\258031084.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory=persistent_directory,


In [7]:
# Define the embedding model
embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/multi-qa-mpnet-base-dot-v1")

C:\Users\Siddharth\AppData\Local\Temp\ipykernel_32664\2525073883.py:2: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/multi-qa-mpnet-base-dot-v1")
c:\Users\Siddharth\.conda\envs\repgpt\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Load the existing vector store with the embedding function
db = Chroma(persist_directory=persistent_directory,
            embedding_function=embeddings)

In [9]:
# Create a retriever for querying the vector store
# `search_type` specifies the type of search (e.g., similarity)
# `search_kwargs` contains additional arguments for the search (e.g., number of results to return)
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)

In [10]:
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, just "
    "reformulate it if needed and otherwise return it as is."
)

In [11]:
# Create a prompt template for contextualizing questions
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


In [12]:
# Create a history-aware retriever
# This uses the LLM to help reformulate the question based on chat history
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


In [13]:
# Answer question prompt
# This system prompt helps the AI understand that it should provide concise answers
# based on the retrieved context and indicates what to do if the answer is unknown
qa_system_prompt = (
    "You are an assistant for question-answering tasks. Use "
    "the following pieces of retrieved context to answer the "
    "question. If you don't know the answer, just say that you "
    "don't know. Use three sentences maximum and keep the answer "
    "concise."
    "\n\n"
    "{context}"
)

In [14]:
# Create a prompt template for answering questions
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [15]:
# Create a chain to combine documents for question answering
# Create stuff_Document_chain feeds all retrieved context into llm
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)


In [16]:
# Create a retreival chain that combines the history aware retriever and qa chain
rag_chain = create_retrieval_chain(
    history_aware_retriever, question_answer_chain
)

In [17]:
# Set Up ReAct Agent with Document Store Retriever
# Load the ReAct Docstore Prompt
react_docstore_prompt = hub.pull("hwchase17/react")

c:\Users\Siddharth\.conda\envs\repgpt\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [18]:
tools = [(
    Tool(
        name="Answer question",
        func = lambda input, **kwargs: rag_chain.invoke(
            {
                "input": input, "chat_history": kwargs.get("chat_history, []")
            }
        ),
        description="useful for when you need to answer question about the context"
    )
)]

In [19]:
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=react_docstore_prompt
)

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, handle_parsing_errors=True, verbose=True
)

In [30]:
#from langchain_core.messages.utils import convert_to_messages

In [25]:
chat_history = []
while True:
    query = input("You: ")
    if query.lower() == "exit":
        break
    response = agent_executor.invoke(
        {"input": query, "chat_history": chat_history})
    print(f"AI: {response['output']}")

    # Update history
    chat_history.append(HumanMessage(content=query))
    chat_history.append(AIMessage(content=response["output"]))
